# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Liu  ->  J. Liu  |  ['J. Liu']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
Y. Khusanova  ->  Y. Khusanova  |  ['Y. Khusanova']
F. Walter  ->  F. Walter  |  ['F. Walter']
M. Ramirez  ->  M. Ramirez-Tannus  |  ['M. Ramirez']
Arxiv has 50 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2408.12104


extracting tarball to tmp_2408.12104...

 done.
Retrieving document from  https://arxiv.org/e-print/2408.12233


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2715/1211882699.py:51: LatexWarning: 2408.12104 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


...

 done.


Y. Khusanova  ->  Y. Khusanova  |  ['Y. Khusanova']
F. Walter  ->  F. Walter  |  ['F. Walter']


Found 44 bibliographic references in tmp_2408.12299/sample631.bbl.
Retrieving document from  https://arxiv.org/e-print/2408.12393
extracting tarball to tmp_2408.12393...

 done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2715/1211882699.py:51: LatexWarning: 2408.12393 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.12299-b31b1b.svg)](https://arxiv.org/abs/2408.12299) | **Far-Infrared and [CII] observations of a z=7 blazar**  |
|| E. Banados, et al. -- incl., <mark>Y. Khusanova</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2024-08-23*|
|*Comments*| *Version after addressing referee report. See also companion paper: arXiv:2407.07236*|
|**Abstract**|            We present millimeter observations of the host galaxy of the most distant blazar known, VLASSJ041009.05-013919.88 (hereafter J0410-0139) at z=7, using ALMA and NOEMA observations. The ALMA data reveal a 2e42 erg/s [CII] 158um emission line at z=6.9964 with a [CII]-inferred star-formation rate of 58 Msun/yr. We estimate a dynamical mass of 4.6e9 Msun, implying a black hole mass to host a dynamical mass ratio of 0.15. The 238 GHz continuum (rest-frame IR) decreased by ~33% from the NOEMA to the ALMA observations taken ~10 months apart. The VLA 3-10 GHz radio flux densities showed a ~37% decrease in a similar time frame, suggesting a causal connection. At face value, J0410-0139 would have the lowest [CII]-to-IR luminosity ratio of a z>5.7 quasar reported to date (~1e-4). However, if only <20% of the measured IR luminosity were due to thermal emission from dust, the [CII]-to-IR luminosity ratio would be typical of (U)LIRGS, and the star formation rates derived from [CII] and IR luminosities would be consistent. These results provide further evidence that synchrotron emission significantly contributes to the observed rest-frame IR emission of J0410-0139, similar to what has been reported in some radio-loud AGN at z<1.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.12233-b31b1b.svg)](https://arxiv.org/abs/2408.12233) | **Gas dynamics around a Jupiter-mass planet I. Influence of protoplanetary disk properties**  |
|| E. Lega, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2024-08-23*|
|*Comments*| *17 pages, 11 figures*|
|**Abstract**|            Giant planets grow and acquire their gas envelope during the disk phase. At the time of the discovery of giant planets in their host disk, it is important to understand the interplay between the host disk and the envelope and circum-planetary disk properties of the planet. Our aim is to investigate the dynamical and physical structure of the gas in the vicinity of a Jupiter-mass planet and study how protoplanetary disk propertiesy, determine the planetary system as well as the accretion rate inside the planet's Hill sphere.We ran global 3D simulations with the grid-based code fargOCA, using a fully radiative equation of state and a dust-to-gas ratio of 0.01. We explored three models. The nominal one features a disk with surface density, Sigma, corresponding to the MMSN at the planet's location, characterised by an alpha viscosity value of 0.004. The second model has a surface density ten times smaller than the nominal one and the same viscosity. In the third model, we also reduced the viscosity value by a factor of 10. During gap formation gas is heated by compression and cools according to opacity, density, and temperature values. In the analysis of our disks, we find that the gas flowing into the Hill sphere is approximately scaled as the product Sigma nu , as expected from viscous transport while The accretion rate is scaled as sqrt(Sigma nu ).Previous studies have shown that pressure or rotationally supported structures are formed around giant planets, depending on the equation of state (EoS) or on the opacity. In the case of a fully radiative EoS and a constant dust to gas ratio of 0.01, we find that low-mass and low-viscosity circum-stellar disks favour the formation of a rotationally supported circum-planetary disk. Gas accretion leading to the doubling time of the planetary system >10^5 years has only been found in the case of a low-viscosity disk.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.12104-b31b1b.svg)](https://arxiv.org/abs/2408.12104) | **Minute-Cadence Observations of the LAMOST Fields with the TMTS: IV -- Catalog of Cataclysmic Variables from the First 3-yr Survey**  |
|| Q. Liu, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-08-23*|
|*Comments*| *27 pages, 12 figures in main text, accepted for the publication in Universe*|
|**Abstract**|            The Tsinghua University--Ma Huateng Telescopes for Survey (TMTS) started to monitor the LAMOST plates in 2020, leading to the discovery of numerous short-period eclipsing binaries, peculiar pulsators, flare stars, and other variable objects. Here, we present the uninterrupted light curves for a sample of 64 cataclysmic variables (CVs) observed/discovered using the TMTS during its first three-year observations, and we introduce new CVs and new light-variation periods (from known CVs) revealed through the TMTS observations. Thanks to the high-cadence observations of TMTS, diverse light variations, including superhumps, quasi-periodic oscillations, large-amplitude orbital modulations, and rotational modulations, are able to be detected in our CV samples, providing key observational clues for understanding the fast-developing physical processes in various CVs. All of these short-timescale light-curve features help further classify the subtypes of CV systems. We highlight the light-curve features observed in our CV sample and discuss further implications of minute-cadence light curves for CV identifications and classifications. Moreover, we examine the H$\alpha$ emission lines in the spectra from our nonmagnetic CV samples (i.e., dwarf novae and nova-like subclasses) and find that the distribution of H$\alpha$ emission strength shows significant differences between the sources with orbital periods above and below the period gap, which agrees with the trend seen from the SDSS nonmagnetic CV sample.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.12393-b31b1b.svg)](https://arxiv.org/abs/2408.12393) | **Massive stars exploding in a He-rich circumstellar medium $-$ X. Flash spectral features in the Type Ibn SN 2019cj and observations of SN 2018jmt**  |
|| Z.-Y. Wang, et al. -- incl., <mark>M. Ramirez</mark> |
|*Appeared on*| *2024-08-23*|
|*Comments*| *28 pages, 19 figures, Accepted for publication in Astronomy & Astrophysics*|
|**Abstract**|            We present optical and near-infrared observations of two Type Ibn supernovae (SNe), SN 2018jmt and SN 2019cj. Their light curves have rise times of about 10 days, reaching an absolute peak magnitude of $M_g$(SN 2018jmt) = $-$19.07 $\pm$ 0.37 and $M_V$(SN 2019cj) = $-$18.94 $\pm$ 0.19 mag, respectively. The early-time spectra of SN 2018jmt are dominated by a blue continuum, accompanied by narrow (600$-$1000 km~s$^{-1}$) He I lines with P-Cygni profile. At later epochs, the spectra become more similar to those of the prototypical SN Ibn 2006jc. At early phases, the spectra of SN 2019cj show flash ionisation emission lines of C III, N III and He II superposed on a blue continuum. These features disappear after a few days, and then the spectra of SN 2019cj evolve similarly to those of SN 2018jmt. The spectra indicate that the two SNe exploded within a He-rich circumstellar medium (CSM) lost by the progenitors a short time before the explosion. We model the light curves of the two SNe Ibn to constrain the progenitor and the explosion parameters. The ejecta masses are consistent with either that expected for a canonical SN Ib ($\sim$ 2 M$_{\odot}$) or those from a massive WR star ($>$ $\sim$ 4 M$_{\odot}$), with the kinetic energy on the order of $10^{51}$ erg. The lower limit on the ejecta mass ($>$ $\sim$ 2 M$_{\odot}$) argues against a scenario involving a relatively low-mass progenitor (e.g., $M_{ZAMS}$ $\sim$ 10 M$_{\odot}$). We set a conservative upper limit of $\sim$0.1 M$_{\odot}$ for the $^{56}$Ni masses in both SNe. From the light curve modelling, we determine a two-zone CSM distribution, with an inner, flat CSM component, and an outer CSM with a steeper density profile. The physical properties of SN 2018jmt and SN 2019cj are consistent with those expected from the core collapse of relatively massive, stripped-envelope (SE) stars.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2408.12299/./figures/f1.png', 'tmp_2408.12299/./figures/f3.png', 'tmp_2408.12299/./figures/f4.png']
copying  tmp_2408.12299/./figures/f1.png to _build/html/
copying  tmp_2408.12299/./figures/f3.png to _build/html/
copying  tmp_2408.12299/./figures/f4.png to _build/html/
exported in  _build/html/2408.12299.md
    + _build/html/tmp_2408.12299/./figures/f1.png
    + _build/html/tmp_2408.12299/./figures/f3.png
    + _build/html/tmp_2408.12299/./figures/f4.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\blazar}{J0410--0139}$
$\newcommand{\lya}{\ensuremath{\text{Ly}\alpha}}$
$\newcommand{\nv}{\ensuremath{\text{N} \textsc{v}}}$
$\newcommand{\lyaew}{\mathrm{EW}(\lya + \nv)}$
$\newcommand{\hi}{\ensuremath{\text{H} \textsc{i}}}$
$\newcommand{\kms}{{\rm km s}\ensuremath{^{-1}}}$
$\newcommand\aliii{\ensuremath{\text{Al \textsc{iii}}}}$
$\newcommand\siiii{\ensuremath{\text{Si \textsc{iii]}}}}$
$\newcommand\siiv{\ensuremath{\text{Si \textsc{iv}}}}$
$\newcommand\civ{\ensuremath{\text{C \textsc{iv}}}}$
$\newcommand\mgii{\ensuremath{\text{Mg \textsc{ii}}}}$
$\newcommand\heii{\ensuremath{\text{He \textsc{ii}}}}$
$\newcommand\feii{\ensuremath{\text{Fe \textsc{ii}}}}$
$\newcommand\feiii{\ensuremath{\text{Fe \textsc{iii}}}}$
$\newcommand\cii{\ensuremath{\text{[C \textsc{ii]}}}}$
$\newcommand\ciii{\ensuremath{\text{C \textsc{iii]}}}}$
$\newcommand\ciilong{\ensuremath{\text{[C \textsc{ii]}}_{158 \mu\text{m}}}}$
$\newcommand\heii{\ensuremath{\text{He \textsc{ii}}}}$
$\newcommand\oiiibr{\ensuremath{\text{O \textsc{iii]}}}}$
$\newcommand\halpha{\ensuremath{\text{H}\alpha}}$
$\newcommand{\km}{{\rm km}}$
$\newcommand{\kmsdeg}{{\rm km s^{-1} deg^{-1}}}$
$\newcommand{\kmskpc}{{\rm km s^{-1} kpc^{-1}}}$
$\newcommand{\mas}{{\rm mas}}$
$\newcommand{\masyr}{{\rm mas/yr}}$
$\newcommand{\kpc}{{\rm kpc}}$
$\newcommand{\kpcdeg}{{\rm kpc deg^{-1}}}$
$\newcommand{\mpc}{{\rm Mpc}}$
$\newcommand{\hmpc}{{h^{-1} \rm Mpc}}$
$\newcommand{\hkpc}{{h^{-1} \rm kpc}}$
$\newcommand{\lnu}{{\rm erg s^{-1} Hz^{-1}}}$</div>



<div id="title">

# [CII] properties and Far-Infrared variability of a $z=7$ blazar

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.12299-b31b1b.svg)](https://arxiv.org/abs/2408.12299)<mark>Appeared on: 2024-08-23</mark> -  _Version after addressing referee report. See also companion paper: arXiv:2407.07236_

</div>
<div id="authors">

E. Ba{ñ}ados, et al. -- incl., <mark>Y. Khusanova</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We present millimeter observations of the host galaxy of the most distant blazar known, VLASS J041009.05 $-$ 013919.88 (hereafter J0410--0139) at $z=7$ , using ALMA and NOEMA observations. The ALMA data reveal a $(2.02\pm 0.36)\times 10^{42}$ erg s $^{-1}$ $\cii$ 158 $\mu$ m emission line at $z=6.9964$ with a $\cii$ -inferred star-formation rate of $58\pm 9 M_\odot $ yr $^{-1}$ .We estimate a dynamical mass of $M_{\rm dyn,\cii}=(4.6\pm 2.0)\times 10^9 M_\odot$ , implying a black hole mass to host a dynamical mass ratio of $0.15^{+0.08}_{-0.05}$ .The 238 GHz continuum (rest-frame IR) decreased by $\sim33\%$ from the NOEMA to the ALMA observations taken $\sim$ 10 months apart.  The VLA 3--10 GHz radio flux densities showed a $\sim37\%$ decrease in a similar time frame, suggesting a causal connection. At face value, $\blazar$ would have the lowest $\cii$ -to-IR luminosity ratio of a $z>5.7$ quasar reported to date ( $\sim10^{-4}$ ). However, if only $<20\%$ of the measured IR luminosity were due to thermal emission from dust, the $\cii$ -to-IR luminosity ratio would be typical of (U)LIRGS, and the star formation rates derived from $\cii$ and IR luminosities would be consistent.These results provide further evidence that synchrotron emission significantly contributes to the observed rest-frame IR emission of $\blazar$ , similar to what has been reported in some radio-loud AGN at $z<1$ .

</div>

<div id="div_fig1">

<img src="tmp_2408.12299/./figures/f1.png" alt="Fig1" width="100%"/>

**Figure 1. -** \small**ALMA and NOEMA observations of $\blazar$**. _Top:_ Continuum-subtracted \cii spectrum, extracted from a $1$\farcs$5$ radius aperture (yellow circle in the next panel). The red line is a Gaussian fit to the data (a second Gaussian can be fitted at $\sim300$ km s$^{-1}$, but we found this not to be statistically significant; see Section \ref{sec:ciiprop}). The \cii measured properties, luminosity, and star-formation rate are listed in Table \ref{tab:properties}. The \cii line is not detected in the NOEMA data.
_Middle:_ ALMA \cii(left) and continuum (right) maps. The beam size for these observations is $0$\farcs$49\times 0$\farcs$68$.  The \cii map is the average over 247 $\kms$(1.2$\times$ FWHM of the spectrum shown in the top panel).
The yellow circle is the $1$\farcs$5$ radius aperture centered on the \cii centroid (yellow cross) used to extract the \cii spectrum.
_Bottom:_ NOEMA continuum maps at 224 (left) and 239 (right) GHz, with beam sizes: $0$\farcs$78\times 1$\farcs$63$(left) and $0$\farcs$72\times 1$\farcs$53$(right). The source is unresolved in the NOEMA observations.
The optical position of the quasar is shown with a white cross on each panel, which is consistent with the peak of the ALMA and NOEMA continuum, but the peak of the \cii map is offset by $0$\farcs$35$(yellow cross in \cii map).
All contour plots are shown at $\pm[3, 5, 10, 20, 30]\sigma$(dashed contours represent negative values). The $\sigma$ for each map is at the top left corner of each panel.
 (*fig:alma_noema*)

</div>
<div id="div_fig2">

<img src="tmp_2408.12299/./figures/f3.png" alt="Fig3" width="100%"/>

**Figure 3. -** \small**Spectral energy distribution (SED) in the radio and (sub)mm regime of $\blazar$**. The data points are color-coded by year of observation for visualization purposes. Only the observations taken at a similar epoch in 2021 and 2022 are shown (see Fig. 2 in  ([Bañados, Momjian and Connor 2024]())  for the full SED and other epochs). The VLA 2021 and 2022 observations were taken in the same observing block, yielding a quasi-simultaneous view of the SED  (within 15 minutes in the rest frame).
The VLA 2021 and NOEMA observations are taken 6.5 days apart in the source's rest frame, while the VLA 2022 and ALMA observations are 2.8 days apart. The dashed lines are power-law extrapolations from the VLA low and high-frequency measurements. The pentagons and star show the NOEMA and ALMA measurements, respectively (see also Fig. \ref{fig:alma_noema}). The $\sim 238 $GHz continuum decreased by $\sim$33\% from the NOEMA 2021 to ALMA 2022 measurements. The VLA flux densities above 3 GHz decreased on average by $\sim 37\%$ from 2021 to 2022 (see Sect. \ref{sec:variability}), strongly suggesting that the differences between the NOEMA and ALMA data are due to the variability of the synchrotron emission.
In contrast, we show models of modified black-body emission (with typical dust temperatures of $T_{\rm d}=47 $K and emissivity index of $\beta=1.6$) with the colored curves. By failing to account for the IR contribution of the synchrotron emission, single-epoch measurements of cold gas in the host galaxy would be very different---and incorrect.
 (*fig:sed*)

</div>
<div id="div_fig3">

<img src="tmp_2408.12299/./figures/f4.png" alt="Fig4" width="100%"/>

**Figure 4. -** **Ratio of [CII] to IR luminosities ($8-1000 \mu$m) vs. IR luminosities.  ** The local galaxies and the high-redshift sources that are _not_ quasars are compiled as in [Bañados, Decarli and Walter (2015)](), including data from [Malhotra, Kaufman and Hollenbach (2001)](), [Luhman, Satyapal and Fischer (2003)](), [Díaz-Santos, Armus and Charmandaris (2013)](), [Cormier and Lebouteiller (2014)](). The $z>5.7$ radio-quiet quasars are from [Venemans, Walter and Neeleman (2020)]() and Wang et al. (2024). The  FIR ($42.5-122.5 \mu$m) luminosities from [Venemans, Walter and Neeleman (2020)]() are converted as $L_{\rm TIR}= 1.41\times L_{\rm FIR}$.
The $z>5.7$ radio-loud quasars are from [Khusanova, Bañados and Mazzucchelli (2022)]() and [Bañados, Schindler and Venemans (2023)](). The open circles are sources undetected in both \cii and IR continuum.
The red star and blue pentagon correspond to the properties of $\blazar$ using the IR properties derived from NOEMA in 2021 and ALMA in 2022, respectively. At face value, these measurements would place $\blazar$ as the lowest data point in $L_{\rm \cii}/L_{\rm IR}$ among the $z>5.7$ quasars. However, we argue that the IR luminosity is contaminated by synchrotron emission, as evidenced by variability. The open star and pentagons show the location of $\blazar$, consistent with local (U)LIRGs, if we require that the SFR derived from IR luminosity matches the one derived from \cii line. In this case, the measured IR luminosity is dominated by synchrotron radiation: $\sim 85\%$ in 2021 and $\sim 80\%$ in 2022.
 (*fig:ciidefilcit*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.12299"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

166  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
